# Image Pattern Generator

This notebook lets you create images by overlaying a smaller image repeatedly on a larger canvas in various patterns (circles, grids, spirals, etc.).

## Setup and Imports

In [ ]:
from pattern_utils import *
from pathlib import Path

# Create directories if they don't exist
Path('input').mkdir(exist_ok=True)
Path('output').mkdir(exist_ok=True)

# Enable inline plotting
%matplotlib inline

# Set figure size for better visualization
plt.rcParams['figure.figsize'] = (10, 10)

print("Setup complete! All pattern generation functions loaded from pattern_utils.py")

## Example 1: Grid Pattern

In [ ]:
# Create a canvas
canvas = create_canvas(1024, 1024, 'white')

# Create a sample small image (black circle)
small_img = create_sample_small_image(width=20, height=20, shape='circle', color=(0, 0, 0, 255))

# Generate grid positions
positions = generate_grid_positions(
    canvas_width=1024,
    canvas_height=1024,
    spacing_x=50,
    spacing_y=50,
    offset_x=25,
    offset_y=25
)

# Visualize positions
visualize_positions(1024, 1024, positions, 'Grid Pattern')

# Overlay the images
result = overlay_images(canvas, small_img, positions, center_images=True)

# Display result
display_image(result, 'Grid Pattern Result')

# Save result
result.save('output/grid_pattern.png')
print("Saved to output/grid_pattern.png")

## Example 2: Single Circle Pattern

In [ ]:
# Create a canvas
canvas = create_canvas(1024, 1024, 'white')

# Create a sample small image (red square)
small_img = create_sample_small_image(width=15, height=15, shape='square', color=(255, 0, 0, 255))

# Generate circle positions
positions = generate_circle_positions(
    center_x=512,
    center_y=512,
    radius=300,
    num_points=50
)

# Visualize positions
visualize_positions(1024, 1024, positions, 'Circle Pattern')

# Overlay the images
result = overlay_images(canvas, small_img, positions, center_images=True)

# Display result
display_image(result, 'Circle Pattern Result')

# Save result
result.save('output/circle_pattern.png')
print("Saved to output/circle_pattern.png")

## Example 3: Concentric Circles Pattern

In [ ]:
# Create a canvas
canvas = create_canvas(1024, 1024, 'white')

# Create a sample small image (blue triangle)
small_img = create_sample_small_image(width=12, height=12, shape='triangle', color=(0, 0, 255, 255))

# Generate concentric circles positions
positions = generate_concentric_circles_positions(
    center_x=512,
    center_y=512,
    min_radius=100,
    max_radius=450,
    num_circles=6,
    points_per_circle=40
)

# Visualize positions
visualize_positions(1024, 1024, positions, 'Concentric Circles Pattern')

# Overlay the images
result = overlay_images(canvas, small_img, positions, center_images=True)

# Display result
display_image(result, 'Concentric Circles Pattern Result')

# Save result
result.save('output/concentric_circles_pattern.png')
print("Saved to output/concentric_circles_pattern.png")

## Example 4: Spiral Pattern

In [ ]:
# Create a canvas
canvas = create_canvas(1024, 1024, 'white')

# Create a sample small image (green circle)
small_img = create_sample_small_image(width=10, height=10, shape='circle', color=(0, 255, 0, 255))

# Generate spiral positions
positions = generate_spiral_positions(
    center_x=512,
    center_y=512,
    start_radius=20,
    end_radius=450,
    num_points=200,
    num_rotations=5
)

# Visualize positions
visualize_positions(1024, 1024, positions, 'Spiral Pattern')

# Overlay the images
result = overlay_images(canvas, small_img, positions, center_images=True)

# Display result
display_image(result, 'Spiral Pattern Result')

# Save result
result.save('output/spiral_pattern.png')
print("Saved to output/spiral_pattern.png")

## Example 5: Square Outline Pattern

In [ ]:
# Create a canvas
canvas = create_canvas(1024, 1024, 'white')

# Create a sample small image (purple square)
small_img = create_sample_small_image(width=18, height=18, shape='square', color=(128, 0, 128, 255))

# Generate square outline positions
positions = generate_square_outline_positions(
    center_x=512,
    center_y=512,
    side_length=600,
    points_per_side=30
)

# Visualize positions
visualize_positions(1024, 1024, positions, 'Square Outline Pattern')

# Overlay the images
result = overlay_images(canvas, small_img, positions, center_images=True)

# Display result
display_image(result, 'Square Outline Pattern Result')

# Save result
result.save('output/square_outline_pattern.png')
print("Saved to output/square_outline_pattern.png")

## Custom Experiment Area

Use this cell to create your own patterns! Mix and match different pattern generators, colors, and shapes.

**Available functions:**
- `create_canvas(width, height, color)`
- `create_sample_small_image(width, height, shape, color)`
- `load_small_image(path)` - for loading your own images
- `generate_grid_positions(...)`
- `generate_circle_positions(...)`
- `generate_concentric_circles_positions(...)`
- `generate_spiral_positions(...)`
- `generate_square_outline_positions(...)`
- `overlay_images(canvas, small_img, positions, center_images=True)`
- `visualize_positions(width, height, positions, title)`
- `display_image(img, title)`

In [ ]:
# Your custom pattern here!
canvas = create_canvas(1024, 1024, 'white')

# Try different parameters:
# - Change the shape: 'circle', 'square', 'triangle'
# - Change the color: (R, G, B, Alpha) where values are 0-255
# - Change the size
small_img = create_sample_small_image(width=10, height=20, shape='circle', color=(0, 0, 0, 255))

# Try different patterns:
# positions = generate_grid_positions(1024, 1024, 30, 30)
# positions = generate_circle_positions(512, 512, 300, 50)
# positions = generate_concentric_circles_positions(512, 512, 100, 450, 5, 30)
# positions = generate_spiral_positions(512, 512, 20, 450, 200, 5)
# positions = generate_square_outline_positions(512, 512, 600, 40)

positions = generate_circle_positions(512, 512, 300, 50)

# Visualize and create
visualize_positions(1024, 1024, positions, 'Custom Pattern')
result = overlay_images(canvas, small_img, positions, center_images=True)
display_image(result, 'Custom Pattern Result')

# Uncomment to save:
# result.save('output/custom_pattern.png')
# print("Saved to output/custom_pattern.png")

## Using Your Own Images

Place your PNG images in the `input/` folder and load them here.

In [ ]:
# Example: Load your own image
# Make sure to place your image in the input/ folder first

# Uncomment and modify the path:
# small_img = load_small_image('input/your_image.png')
# canvas = create_canvas(1024, 1024, 'white')
# positions = generate_circle_positions(512, 512, 300, 50)
# result = overlay_images(canvas, small_img, positions, center_images=True)
# display_image(result, 'Custom Image Pattern')
# result.save('output/custom_image_pattern.png')

print("Place your images in the input/ folder and uncomment the code above to use them!")